<a href="https://colab.research.google.com/github/abhi1ranjan/RL-agorithm-practise/blob/main/RL_algorithms_practise.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Q-Learning implementation using gym environment**

In [ ]:
import gym
import numpy as np

In [ ]:
# Load the environment
env = gym.make('FrozenLake-v0')

Implement the Q-Learning algorithm

In [ ]:
# Initialize the table with all zeroes to maintain uniformity
QTable = np.zeros([env.observation_space.n,env.action_space.n]) 

# learning parameters values
LRate = 0.5
discountFactor = 0.95
totalEpisodes = 2000

# Maintain lists to store the total rewards and steps per episode
Jlist = []
Rlist = []

for itr in range(totalEpisodes):
  #reset the environment and get the new observation
  s = env.reset()
  rAll = 0
  d = False
  j = 0     # denote steps

  # The Q-learning table algorithm
  while j<99:
    j=+1
    # action will be chosen in greedy manner and will picked from Q-table
    action = np.argmax(QTable[s,:] + np.random.randn(1,env.action_space.n)*(1./(itr+1)))
    # Fetch new state and reward from the environment
    s1,r,d,_ = env.step(action)
    # update Q-table using the bellman equation with the help of new information just obtained
    QTable[s,action] = QTable[s,action] + LRate*(r + discountFactor*np.max(QTable[s1,:])-QTable[s,action])
    rAll += r
    s = s1
    if d == True:
      # print("hi")
      break
  Jlist.append(j)
  Rlist.append(rAll)


In [ ]:
Rlist

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0

In [ ]:
QTable.shape

(16, 4)

In [ ]:
print("score over time instance: " + str(sum(Rlist)/totalEpisodes))

score over time instance: 0.216


In [ ]:
print("Final Q-table values \n", QTable) #when score is 0.216

Final Q-table values 
 [[3.38182689e-02 3.90756950e-03 4.02887492e-03 4.11781779e-03]
 [2.15629456e-03 2.35644754e-03 2.69104789e-03 4.34198917e-03]
 [3.72065078e-03 4.02285255e-03 0.00000000e+00 4.87726756e-03]
 [2.05264354e-03 3.08794193e-03 3.28312772e-03 6.79860988e-03]
 [4.29767935e-02 2.09511137e-03 1.90645553e-03 1.31890605e-03]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [3.25318047e-03 4.26739646e-04 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 1.65924208e-03 5.57885940e-02]
 [2.32868389e-03 7.02337085e-02 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 1.88831445e-01 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 0.00000000e+00]
 [1.37854939e-04 2.33452984e-04 2.75709879e-04 1.72049042e-01]
 [0.00000000e+00 0.00000000e+00 0.00000000e+00 4.86453791e-01]
 [0.00000000e+00 0.00000000e+00 

In [ ]:
print("Final Q-table values", QTable)  # when score is 0.28

Final Q-table values [[0.01746332 0.10590996 0.01547394 0.01658849]
 [0.0006966  0.00858767 0.00465219 0.06451269]
 [0.00293171 0.00481483 0.03029571 0.00265921]
 [0.         0.00129215 0.00214694 0.06509362]
 [0.13153378 0.0066275  0.00434676 0.01517411]
 [0.         0.         0.         0.        ]
 [0.         0.03302458 0.00209638 0.0011909 ]
 [0.         0.         0.         0.        ]
 [0.01080973 0.0063861  0.00203285 0.16350595]
 [0.00835711 0.16594574 0.0099972  0.        ]
 [0.11156529 0.00128714 0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.0010198  0.         0.         0.08728638]
 [0.         0.48049488 0.         0.        ]
 [0.         0.         0.         0.        ]]


**Tensorflow implementation of simple Q-Network learning**

In [81]:
import gym
import numpy as np
import random
import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
%matplotlib inline

In [54]:
# Load the environment

env = gym.make('FrozenLake-v0')

implementing the network itself

In [82]:
# tf.reset_default_graph()
tf.compat.v1.reset_default_graph()

In [83]:
# establishing the feed-forward part of the network used to choose actions

# tf.disable_v2_behavior()
tf.compat.v1.disable_eager_execution()
inputs1 = tf.compat.v1.placeholder(shape = [1,16],dtype = tf.float32)
W = tf.Variable(tf.random.uniform([16,4],0,0.01))
QOutput = tf.matmul(inputs1,W)
predict = tf.argmax(QOutput,1)

In [85]:
# Find the loss using the loss function which is sum of square difference between the target and predicted Q values

NextQ = tf.compat.v1.placeholder(shape = [1,4], dtype = tf.float32)
loss = tf.reduce_sum(tf.square(NextQ - QOutput))
# trainer = tf.optimizers.SGD(learning_rate = 0.2)
trainer = tf.train.GradientDescentOptimizer(learning_rate = 0.2)
# tf.disable_v2_behavior()
updateModel = trainer.minimize(loss)

Training the network